In [1]:
%%bash
pip install --no-cache-dir -qU pip wheel
pip install --no-cache-dir -qU numpy pandas plotly dash
pip check

No broken requirements found.


In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go

import dash
from dash import dcc, html

## Plotly figures

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
ecom_sales = ecom_sales.groupby(['Year-Month','Country'])\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')

# Create the line graph
line_graph = px.line(
    # Set the appropriate DataFrame and title
    data_frame=ecom_sales, title='Total Sales by Country and Month', 
    # Set the x and y arguments
    x='Year-Month', y='Total Sales ($)',
    # Ensure a separate line per country
    color='Country'
)
line_graph.show()

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
ecom_sales = ecom_sales.groupby('Country')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')

# Create the bar graph object
bar_fig = px.bar(
  # Set the DataFrame, x and y
  data_frame=ecom_sales, x='Total Sales ($)', y='Country',
  # Set the graph to be horizontal
  orientation='h', title='Total Sales by Country'
)

# Increase the gap between bars
bar_fig.update_layout({'bargap': 0.5})

bar_fig.show()

## Dash components

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
ecom_sales = ecom_sales.groupby(['Year-Month','Country'])\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
line_fig = px.line(data_frame=ecom_sales, x='Year-Month', y='Total Sales ($)', 
    title='Total Sales by Month', color='Country')

# Create the Dash app
app = dash.Dash()

# Set up the layout with a single graph
app.layout = dcc.Graph(
    id='my-line-graph',
    # Insert the line graph
    figure=line_fig)

# Set the app to run in development mode
if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
ecom_line = ecom_sales.groupby(['Year-Month','Country'])\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
ecom_bar = ecom_sales.groupby('Country')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
line_graph = px.line(data_frame=ecom_line, x='Year-Month', y='Total Sales ($)',
    title='Total Sales by Month', color='Country')
bar_graph = px.bar(data_frame=ecom_bar, x='Total Sales ($)', y='Country',
    orientation='h',title='Total Sales by Country')

# Create the Dash app
app = dash.Dash()

# Set up the layout using an overall div
app.layout = html.Div(children=[
    # Add a H1
    html.H1('Sales by Country & Over Time'),
    # Add both graphs
    dcc.Graph(id='line_graph', figure=line_graph),
    dcc.Graph(id='bar_graph', figure=bar_graph)
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
ecom_line = ecom_sales.groupby('Year-Month')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
line_fig = px.line(data_frame=ecom_line, x='Year-Month', y='Total Sales ($)',
    title='Total Sales by Month')
ecom_bar = ecom_sales.groupby('Country')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
max_country = ecom_bar.sort_values(by='Total Sales ($)', ascending=False)\
    .loc[0]['Country']
bar_fig = px.bar(data_frame=ecom_bar, x='Total Sales ($)', y='Country',
    orientation='h', title='Total Sales by Country')

app = dash.Dash()

# Create the dash layout and overall div
app.layout = html.Div(children=[
    html.H1('Sales Figures'), 
    # Add a div containing the line figure
    html.Div(dcc.Graph(id='my-line-fig', figure=line_fig)), 
    # Add a div containing the bar figure
    html.Div(dcc.Graph(id='my-bar-fig', figure=bar_fig)), 
    # Add the H3
    html.H3(f'The largest country by sales was {max_country}')
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
bar_fig_country = px.bar(
    data_frame=ecom_sales, x='Total Sales ($)', y='Country', color='Country',
    color_discrete_map={
        'United Kingdom': 'lightblue',
        'Germany': 'orange',
        'France': 'darkblue',
        'Australia': 'green',
        'Hong Kong': 'red'}
)
app = dash.Dash()
app.layout = html.Div(children=[
    html.H1('Sales Proportion by Country'),
    dcc.Graph(id='bar_graph', figure=bar_fig_country),
    html.Div(
        style={'background-color': 'red', 'height': 250, 'width': 250}
    ),
    html.Div(
        children=[
            html.H1('This box'),
            html.H2('Another Title')
        ],
        style={'background-color': 'lightblue'})
])

## HTML

In [ ]:
app.layout = html.Div(children=[
    html.Img(src='www.website.com/logo.png'),
    html.H1('Our Sales Dashboard'),
    html.Span(children=[
        f"Prepared: {datetime.now().date()}",
        html.Br(), " by ", html.B("Meng Oon, "),
        html.Br(), html.I("Data Scientist")
    ])
])

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
logo_link = './Data/e_com_logo.png'
ecom_bar = ecom_sales.groupby('Country')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')\
    .sort_values(by='Total Sales ($)', ascending=False)
top_country = ecom_bar.loc[0]['Country']            
bar_fig_country = px.bar(ecom_bar, x='Total Sales ($)', y='Country',
    color='Country', color_discrete_map={
        'United Kingdom':'lightblue', 'Germany':'orange', 'France':'darkblue',
        'Australia':'green', 'Hong Kong':'red'}
)         

app = dash.Dash()
app.layout = html.Div([
    # Add the company logo
    html.Img(src=logo_link),
    html.H1('Sales by Country'),
    html.Div(dcc.Graph(figure=bar_fig_country), 
        style={'width':'750px', 'margin':'auto'}),
    # Add an overall text-containing component
    html.Span(
        children=[
            # Add the top country text
            'This year, the most sales came from: ', 
            html.B(top_country),
            # Italicize copyright notice
            html.I(' Copyright E-Com INC')
        ])
    ],
    style={'text-align':'center', 'font-size':22}
)

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
logo_link = './Data/e_com_logo.png'
ecom_category = ecom_sales.groupby(['Major Category','Minor Category'])\
    .size().reset_index(name='Total Orders')\
    .sort_values(by='Total Orders', ascending=False)\
    .reset_index(drop=True)
num1_cat, num1_salesvol = ecom_category.loc[0].tolist()[1:3]
num2_cat, num2_salesvol = ecom_category.loc[1].tolist()[1:3]
ecom_bar = px.bar(ecom_category, x='Total Orders', y='Minor Category',
    color='Major Category')
ecom_bar.update_layout({'yaxis':{'dtick':1, 'categoryorder':'total ascending'}})             

app = dash.Dash()
app.layout = html.Div([
    html.Img(src=logo_link),
    html.H1('Top Sales Categories'),
    html.Div(dcc.Graph(figure=ecom_bar)),
    html.Span(
        children=[
            'The top 2 sales categories were:',
            # Set up an ordered list
            html.Ol(
                children=[
                    # Add two list elements with the top category variables
                    html.Li(children=[num1_cat, ' with ', num1_salesvol, ' sales volume']),
                    html.Li(children=[num2_cat, ' with ', num2_salesvol, ' sales volume'])
                ],
            style={'width':'350px', 'margin':'auto'}),
            # Add a line break before the copyright notice
            html.Br(),
            html.I('Copyright E-Com INC')
        ])
    ],
    style={'text-align':'center', 'font-size':22}
)

if __name__ == '__main__':
    app.run_server(debug=True)

## CSS

In [ ]:
app.layout = html.Div([
    html.H1('Welcome to the website!'),
    html.H2('Text', style={'font-size': '50px', 'color': 'red'})
    # Add & resize the company logo
    html.Img(src=logo_link,
        # style={'width': '250px', 'height':'250px'}
        style={'width': '50%', 'height': '50%'})
])

In [ ]:
bball_data = pd.read_csv('./Data/bball_data.csv')

# Create and show the plot
fig = px.scatter(
    data_frame=bball_data,
    x='PPG',
    y='FGP',
    title="Field Goal Percentage vs. Points Per Game"
)
fig.show()
# Update the x_axis range
fig.update_layout({
    'xaxis': {
        'range': [0, bball_data['PPG'].max() + 5]
    },
    'yaxis': {
        'range': [0, 100]
    }
})
fig.show()

In [ ]:
penguins = pd.DataFrame({
    'spec': ['Adelie', 'Chinstrap', 'Gentoo'],
    'av_flip_length': [190.0, 196.0, 217.0]
})
# Create timestamp
timestamp = datetime.now()
# Create plot
fig = px.bar(
    data_frame=penguins,
    x='spec',
    y='av_flip_length',
    title='Flipper Length (mm) by Species',
    color='spec'
)
fig.update_layout({
    # Change the axis titles
    'xaxis': {
        'title': {'text': 'Species'}
    },
    'yaxis': {
        'title': {'text': 'Average Flipper Length (mm)'}
    },
    # Add an annotation
    'annotations': [{
        'x': 0.5, 'y': 1.1,
        'xref': 'paper', 'yref': 'paper',
        'text': f"This graph was generated at {timestamp}",
        'showarrow': False
    }]
})
# Show the plot
fig.show()

In [ ]:
revenues = pd.read_csv('./Data/revenue_data.csv')
revenues['Revenue'] = revenues['Revenue'].astype(float)

fig = make_subplots(rows=2, cols=1,
    subplot_titles=['Histogram of company revenues', 
        'Box plot of company revenues'])
## Add in traces (fig.add_trace())
fig.add_trace(
    go.Histogram(x=revenues['Revenue'], nbinsx=5, name='Histogram'),
    row=1, col=1
)
fig.add_trace(
    go.Box(x=revenues['Revenue'], hovertext=revenues['Company'], name='Box plot'),
    row=2, col=1
)
fig.update_layout({'title': 
    {'text': 'Plots of company revenues', 'x': 0.5, 'y': 0.9}
})
fig.show()

In [ ]:
penguins = pd.read_csv('./Data/penguins.csv')

fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
row_num = 1
for species in penguins['Species'].unique():
    df = penguins[penguins['Species']==species]
    fig.add_trace(
        go.Scatter(
            x=df['Culmen Length (mm)'],
            y=df['Culmen Depth (mm)'],
            name=species, mode='markers'),
        row=row_num, col=1
    )
    row_num += 1
fig.show()

In [ ]:
revenues = pd.read_csv('./Data/revenue_data2.csv')
df_tech = revenues[revenues['Industry']=='Tech']
df_prof_serve = revenues[revenues['Industry']=='Professional Services']
df_retail = revenues[revenues['Industry']=='Retail']
df_oil = revenues[revenues['Industry']=='Oil']
# Set up the subplots grid
fig = make_subplots(rows=2, cols=2,
    # Set the subplot titles
    subplot_titles=['Tech', 'Professional Services', 'Retail', 'Oil']
)
# Add the Tech trace
fig.add_trace(go.Box(x=df_tech.Revenue.astype(float), name='Tech', 
    showlegend=False), row=1, col=1)
# Add the Professional Services trace
fig.add_trace(go.Box(x=df_prof_serve.Revenue.astype(float), name='Professional Services', 
    showlegend=False), row=1, col=2)
# Add the Retail trace
fig.add_trace(go.Box(x=df_retail.Revenue.astype(float), name='Retail', 
    showlegend=False), row=2, col=1)
# Add the Oil trace
fig.add_trace(go.Box(x=df_oil.Revenue.astype(float), name='Oil', 
    showlegend=False), row=2, col=2)
# Add a title (and show)
fig.update_layout({'title': 
    {'text': 'Box plots of company revenues', 'x': 0.5, 'y': 0.9}})
fig.show()

In [ ]:
revenues = pd.read_csv('./Data/revenue_data2.csv')
# Create the subplots
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
# Loop through the industries
row_num = 1
for industry in ['Tech', 'Retail', 'Professional Services']:
    df = revenues[revenues.Industry==industry]
    # Add a histogram using subsetted df
    fig.add_trace(
        go.Histogram(x=df['Revenue'].astype(float), name=industry),
        # Position the trace
        row=row_num, col=1)
    row_num += 1
# Show the plot
fig.show()

In [ ]:
penguins_grouped = pd.read_csv('./Data/penguins_grouped.csv')
# Create the base figure
fig = go.Figure()
# Loop through the species
for species in ['Adelie', 'Chinstrap', 'Gentoo']:
    # Add a bar chart trace
    fig.add_trace(go.Bar(
        x=penguins_grouped['Island'].unique(),
        # Set the appropriate y subset and name
        y=penguins_grouped[penguins_grouped.Species==species]['Count'],
        name=species
    ))
# Show the figure
fig.show()

In [ ]:
temp_syd = pd.read_csv('./Data/sydney_temps.csv', 
    parse_dates=['Date'])
temp_syd_avg = pd.read_csv('./Data/sydney_temps_avg.csv', 
    parse_dates=['Date'])
# Create the base figure
fig = go.Figure()
# Add the bar graph of daily temperatures
fig.add_trace(go.Bar(
    x=temp_syd['Date'],
    y=temp_syd['Temp'],
    name='Daily Max Temperature'
))
# Add the monthly average line graph
fig.add_trace(go.Scatter(
    x=temp_syd_avg['Date'],
    y=temp_syd_avg['Average'],
    name='Average Monthly Temperature',
    mode='lines+markers'
))
# Show the plot
fig.show()

In [ ]:
rain = pd.read_csv('./Data/rain.csv', 
    parse_dates=['Date'], dayfirst=True)
# Create the buttons
date_buttons = [
    {'count': 28, 'label': '4WTD', 'step': 'day', 'stepmode': 'todate'},
    {'count': 48, 'label': '48HR', 'step': 'hour', 'stepmode': 'todate'},
    {'count': 1, 'label': 'YTD', 'step': 'year', 'stepmode': 'todate'}
]
# Create the basic line chart
fig = px.line(data_frame=rain, x='Date', y='Rainfall',
    title='Rainfall (mm)')
# Add the buttons and show
fig.update_layout({
    'xaxis': {'rangeselector': {'buttons': date_buttons}}
})
fig.show()

In [ ]:
tesla = pd.read_csv('./Data/AAPL.csv', 
    parse_dates=['Date'])
# Create the basic line chart
fig = px.line(data_frame=tesla, x='Date', y='Open',
    title="Tesla Opening Stock Prices")
# Create the financial buttons
fin_buttons = [
    {'count': 7, 'label': '1WTD', 'step': 'day', 'stepmode': 'todate'},
    {'count': 6, 'label': '6MTD', 'step': 'month', 'stepmode': 'todate'},
    {'count': 1, 'label': 'YTD', 'step': 'year', 'stepmode': 'todate'}
]
# Create the date range buttons & show the plot
fig.update_layout({
    'xaxis': {'rangeselector': {'buttons': fin_buttons}}
})
fig.show()

In [ ]:
revenues = pd.read_csv('./Data/revenue_data2.csv')
revenues = revenues[revenues['Industry'].isin(
    ['Tech', 'Oil', 'Pharmaceuticals', 'Professional Services'])]
revenues['Revenue'] = revenues['Revenue'].astype(float)
# Create a simple bar chart
fig = px.bar(
    data_frame=revenues, x='Industry', y='Revenue',
    color='Industry'
)
# Create the buttons
my_buttons = [
    {'label': 'Bar plot', 'method': 'update', 
        'args': [{'type': 'bar'}]},
    {'label': 'scatterplot', 'method': 'update',
        'args': [{'type': 'scatter', 'mode': 'markers'}]}
]
# Add buttons to the plot and show
fig.update_layout({
    'updatemenus': [{
        'type': 'buttons',
        'direction': 'down',
        'x': 1.3, 'y': 0.5,
        'showactive': True,
        'active': 0,
        'buttons': my_buttons
    }]
})
fig.show()

In [ ]:
sales = pd.read_csv('./Data/monthly_sales.csv', 
    parse_dates=['Month Start'], dayfirst=True)

# Create the basic figure
fig = go.Figure()

# Add a trace per metric
fig.add_trace(go.Bar(
    x=sales['Month'], y=sales['Sales Volume'], name='Sales Volume'
))
fig.add_trace(go.Bar(
    x=sales['Month'], y=sales['Sales Value'], name='Sales Value'
))
# Create annotations
volume_annotations = [{
    'text': 'Oct was the best', 'showarrow': True, 'x': 'October', 'y': 71900
}]
value_annotations = [{
    'text': 'Sept was the best', 'showarrow': True, 'x': 'September', 'y': 345397
}]
# Create buttons
my_buttons = [
    {'label': 'By Sales Value', 'method': 'update', 
        'args': [{}, {'annotations': value_annotations}]},
    {'label': 'By Sales Volume', 'method': 'update',
        'args': [{}, {'annotations': volume_annotations}]}
]
# Add the buttons
fig.update_layout({
    'updatemenus': [{'type': 'buttons', 'direction': 'down',
        'x': 1.3, 'y': 0.5, 'buttons': my_buttons}]
})
# Take a look at plot so far
fig.show()

In [ ]:
# Create the basic figure
fig = go.Figure()
# Loop through the states
pop_growth = pd.read_csv('./Data/pop_growth.csv')
pop_growth['LGA code'] = pop_growth['LGA code'].astype(str)
for state in ['NSW', 'QLD', 'VIC']:
    # Subset the DataFrame
    df = pop_growth[pop_growth['State']==state]
    # Add a trace for each state subset
    fig.add_trace(go.Bar(
        x=df['Local Government Area'], y=df['Change %'], name=state
    ))
# Create the buttons
dropdown_buttons = [
    {'label': 'ALL', 'method': 'update',
        'args': [{'visible': [True, True, True]}, {'title': 'ALL'}]},
    {'label': 'NSW', 'method': 'update',
        'args': [{'visible': [True, False, False]}, {'title': 'NSW'}]},
    {'label': 'QLD', 'method': 'update',
        'args': [{'visible': [False, True, False]}, {'title': 'QLD'}]},
    {'label': 'VIC', 'method': 'update',
        'args': [{'visible': [False, False, True]}, {'title': 'VIC'}]}
]
# Update the figure to add dropdown menu
fig.update_layout({
    'updatemenus': [{
        'type': 'dropdown',
        'x': 1.3,
        'y': 0.5,
        'showactive': True,
        'active': 0,
        'buttons': dropdown_buttons
    }]
})
# Show the plot
fig.show()

In [ ]:
# Create the basic figure
fig = go.Figure()
# Loop through the suburbs
house_prices = pd.read_csv('./Data/house_prices.csv')
for suburb in ['Greenacre', 'Lakemba']:
    # Subset the DataFrame
    df = house_prices[house_prices['Suburb']==suburb]
    # Add a trace for each suburb subset
    fig.add_trace(go.Scatter(
        x=df['Year'],
        y=df['Median House Price'],
        name=suburb, mode='lines'
    ))
# Annotation
ga_annotation = [{
    'text': 'Price boom!', 'showarrow': True,
    'x': 'Year: 2018', 'y': 712678
}]
# Create the buttons
dropdown_buttons = [
    {'label': 'Greenacre', 'method': 'update',
        'args': [{'visible': [True, False]}, 
            {'title': 'Greenacre', 'annotations': ga_annotation}]},
    {'label': 'Lakemba', 'method': 'update',
        'args': [{'visible': [False, True]},
            {'title': 'Lakemba', 'annotations': []}]}
]
fig.update_layout({
    'updatemenus': [{
        'type': 'dropdown', 'x': 1.3, 'y': 0.5,
        'showactive': True, 'active': 0, 'buttons': dropdown_buttons
    }]
})
# Show the plot
fig.show()

In [ ]:
fig = px.scatter(
    data_frame=revenues,
    x='Employees', y='Revenue', color='Industry',
    animation_frame='Year', animation_group='Company'
)
fig.update_layout({
    'xaxis': {'range': [-100000, 2500000]},
    'yaxis': {'range': [0, 500000]}
})
fig['layout'].pop('updatemenus')
fig.show()

In [ ]:
# Create the basic figure
fig = go.Figure()
penguins = pd.read_csv('./Data/penguins.csv')
# Loop through the island
for island in ['Torgersen', 'Biscoe', 'Dream']:
    df = penguins[penguins.Island==island]
    fig.add_trace(go.Scatter(
        x=df['Culmen Length (mm)'],
        y=df['Culmen Depth (mm)'],
        mode='markers', name=island
    ))
# Create the slider elements
sliders = [{
    'steps': [
        {'label': 'Torgersen', 'method': 'update',
            'args': [{'visible': [True, False, False]}]},
        {'label': 'Bisco', 'method': 'update',
            'args': [{'visible': [False, True, False]}]},
        {'label': 'Dream', 'method': 'update',
            'args': [{'visible': [False, False, True]}]}
    ]
}]
# Make traces invisible
fig.data[1].visible = False
fig.data[2].visible = False
# Update the figure to add sliders and show
fig.update_layout({'sliders': sliders})
# Show the plot
fig.show()

In [ ]:
# Create the basic figure
fig = go.Figure()
rain_pm = pd.read_csv('./Data/rain_pm.csv')
# Loop through the seasons
for season in ['Autumn', 'Winter', 'Spring']:
    # Subset the DataFrame
    df = rain_pm[rain_pm['Season']==season]
    # Add a trace for each season
    fig.add_trace(go.Bar(
        x=df['Month'], y=df['Total Rainfall'], name=season
    ))
# Create the slider elements
sliders = [{
    'steps': [
        {'label': 'Autumn', 'method': 'update',
            'args': [{'visible': [True, False, False]}]},
        {'label': 'Winter', 'method': 'update',
            'args': [{'visible': [False, True, False]}]},
        {'label': 'Spring', 'method': 'update',
            'args': [{'visible': [False, False, True]}]}
    ]
}]
# Update the figure to add sliders and show
fig.update_layout({'sliders': sliders})
# Show the plot
fig.show()